In [184]:
import pandas as pd
pd.set_option("display.max_rows", 150)
pd.set_option('display.max_colwidth', -1)

import numpy as np

import glob

import gpxpy
import gpxpy.gpx

import re

from IPython.display import display, HTML

import requests
import pickle

In [185]:
# functions for judging a description
not_flowing_keywords = ["dry", "no water", "no flow", "not "]
def isNotFlowing(description):
    return any([kw in description for kw in not_flowing_keywords])
assert(isNotFlowing("looks dry"))

flowing_keywords = ["good", "flowing", "flow", "lots", "working"]
def isFlowing(description):
    return any([kw in description for kw in flowing_keywords])
assert(isFlowing("flowing well"))

In [186]:
# a function that tries to fix a location id, leaves non-waypoint labels alone in case they are in the gps data
# fixes the leading zeros problem WR004 = WR0004
def clean_loc_id(id):
    try:
        waypoint_search = re.search('((?:WR|WA)(?:CS|))([0-9]+)', id, re.IGNORECASE)

        if waypoint_search:        
            waypoint_type = waypoint_search.group(1)
            waypoint_num = waypoint_search.group(2)

            return f"{waypoint_type}{int(waypoint_num):04}"
    except:
        return id
    return id
assert(clean_loc_id("WR004") == "WR0004")

In [187]:
# functions for finding the last positive result and last negative result
def last_pos(loc_id, date):
    last_pos_date = data[(data.loc_id == loc_id) & (data.status==1) & (data.date < date)].date.max()
    if type(last_pos_date) == float:
        return 1000
    return (date - last_pos_date).days
    
def last_neg(loc_id, date):
    last_neg_date = data[(data.loc_id == loc_id) & (data.status==0) & (data.date < date)].date.max()    
    if type(last_neg_date) == float:
        return 1000
    return (date - last_neg_date).days

In [188]:
# create a lat, lon, and name dictionaries keyed on loc_ids 
waypoints = []
for path in glob.glob("gps/*.gpx"):
    gpx_file = open(path, 'r')
    gpx = gpxpy.parse(gpx_file)
    waypoints += (gpx.waypoints)
    
lat = {clean_loc_id(wp.name):wp.latitude for wp in waypoints}
lon = {clean_loc_id(wp.name):wp.longitude for wp in waypoints}
name = {clean_loc_id(wp.name):wp.description for wp in waypoints}

In [189]:
# a generator for sequentially reading in the data
def data_generator():
    for f in glob.glob("pct_data/*.xlsx"):
        data = pd.read_excel(f).dropna()
        _,cols = data.shape
        if cols == 7:
            data.columns = ["map","mile","loc_id","loc", "description", "date", "user"]
        if cols == 8:
            data.columns = ["map","mile_2","mile","loc_id","loc", "description", "date", "user"]
        yield data

In [190]:
# import and combine data
data = pd.concat(data_generator())

/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [191]:
# clean up the descriptions
data.description = data.description.str.split("\n").str[0].str.lower()

In [192]:
# clean up the lod_ids
data.loc_id = data.loc_id.apply(clean_loc_id)

In [193]:
# create a unix time column
data["unix_time"] = (pd.to_datetime(data["date"], errors='coerce').astype(np.int64)//1000000000)

In [194]:
# create lat and lon columns
data["lat"] = data["loc_id"].map(lat)
data["lon"] = data["loc_id"].map(lon)

In [195]:
# create not_flowing and flowing columns
data["not_flowing"] = data["description"].map(isNotFlowing, na_action="ignore")
data["flowing"] = data["description"].map(isFlowing, na_action="ignore")
data["decision"] = data.flowing|data.not_flowing
data["status"] = data.flowing.astype(float)

In [196]:
# remove duplicate entries (ones with the same date and loc_id)
data = data.drop_duplicates(subset=["date","loc_id"])

In [197]:
# remove all data with no decision
data = data[data.decision]

In [198]:
# create last positive and negative result columns
data["last_pos"] = data.apply(lambda row: last_pos(row.loc_id, row.date), axis=1)
data["last_neg"] = data.apply(lambda row: last_neg(row.loc_id, row.date), axis=1)

In [199]:
# drop data with missing coordinates
data = data.dropna(subset=["lat"])

In [200]:
loc_ids = list(data.groupby("loc_id").nunique().sort_values(by="date")[-50:].loc_id.index)

In [201]:
data = data[data.loc_id.isin(loc_ids)]

In [202]:
data.shape

(1124, 17)

In [203]:
try:
    f = open('rain_memo.pkl', 'rb')
    rain_memo = pickle.load(f)
    f.close()
except:
    rain_memo = {}

def getRainfall(loc_id, unix_time):
    if (loc_id, unix_time) in rain_memo:
        return rain_memo[(loc_id, unix_time)]
    r = requests.get(f"https://api.darksky.net/forecast/840fdc3fb61acc3d4e904978b5ba8dc5/{lat[loc_id]},{lon[loc_id]},{unix_time}")
    rain = r.json()["daily"]["data"][0]["precipIntensity"]
    rain_memo[(loc_id, unix_time)] = rain
    f = open('rain_memo.pkl', 'wb')
    pickle.dump(rain_memo, f)
    f.close() 
    
    return rain

In [208]:
rain_memo

{('WRCS0232', 1559520000): 0.0019,
 ('WRCS0232', 1559433600): 0.0005,
 ('WRCS0232', 1559347200): 0,
 ('WRCS0232', 1559260800): 0,
 ('WRCS0232', 1559174400): 0.0002,
 ('WRCS0232', 1528675200): 0,
 ('WRCS0232', 1528588800): 0,
 ('WRCS0232', 1528502400): 0,
 ('WRCS0232', 1528416000): 0,
 ('WRCS0232', 1528329600): 0,
 ('WRCS0232', 1539648000): 0,
 ('WRCS0232', 1539561600): 0.0001,
 ('WRCS0232', 1539475200): 0.034,
 ('WRCS0232', 1539388800): 0.0007,
 ('WRCS0232', 1539302400): 0.0001,
 ('WRCS0232', 1556236800): 0,
 ('WRCS0232', 1556150400): 0,
 ('WRCS0232', 1556064000): 0,
 ('WRCS0232', 1555977600): 0,
 ('WRCS0232', 1555891200): 0,
 ('WRCS0232', 1554854400): 0,
 ('WRCS0232', 1554768000): 0,
 ('WRCS0232', 1554681600): 0,
 ('WRCS0232', 1554595200): 0.0001,
 ('WRCS0232', 1554508800): 0,
 ('WRCS0232', 1524182400): 0,
 ('WRCS0232', 1524096000): 0,
 ('WRCS0232', 1524009600): 0,
 ('WRCS0232', 1523923200): 0,
 ('WRCS0232', 1523836800): 0,
 ('WRCS0232', 1526428800): 0,
 ('WRCS0232', 1526342400): 0,
 

In [204]:
def getRainSum(loc_id, unix_time, n):
    return 24 * sum([getRainfall(loc_id, unix_time - (24*60*60*day)) for day in range(n)])

In [205]:
data["rain_5"] = data.apply(lambda row: getRainSum(row.loc_id, row.unix_time, 5), axis=1)

In [206]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, BayesianRidge
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.dummy import DummyClassifier

from sklearn.model_selection import train_test_split

import time

In [207]:


out = []
for loc_id in loc_ids:
    inputs = data[data.loc_id == loc_id][["rain_5","last_pos","last_neg"]]
    labels = data[data.loc_id == loc_id]['status']
    X_train, X_test, y_train, y_test = train_test_split(inputs, labels, random_state=0)

    lr = LogisticRegression(C=100, penalty="l1", class_weight="balanced").fit(X_train, y_train)
    svc = SVC(probability=True).fit(X_train, y_train)
    br = BayesianRidge().fit(X_train, y_train)
    rf = RandomForestClassifier().fit(inputs, labels)

    dummy = DummyClassifier().fit(X_train, y_train)
    dummy_majority = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
    
    print(f'Training score {lr.score(X_train, y_train)}')
    print(f'Training score {svc.score(X_train, y_train)}')
    print(f'Training score {rf.score(X_train, y_train)}')


    print()
    print(f'Test score {lr.score(X_test, y_test)}')
    print(f'Test score {svc.score(X_test, y_test)}')
    print(f'Test score {rf.score(X_test, y_test)}')
    print(f'Dummy score {dummy.score(X_test, y_test)}')
    print(f'Most Frequent score {dummy_majority.score(X_test, y_test)}')
    
    prediction = rf.predict_proba([getRainSum(loc_id, int(time.time()), 5), ])
    out.append({'lat': lat[loc_id], 'lon': lon[loc_id], 'name': name[loc_id], 'prediction':prediction})

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1.0